In [1]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import CharDataTransformer, DataTransformer, DataLoader
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS = 5000
EMBEDDING_DIM = 150
MAX_SEQUENCE_LENGTH = 50
OUT_SIZE = 1

In [3]:
data_transformer = CharDataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains, tests, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))
labels = to_categorical(labels)

[DataHelper] Apply normalization on value-type columns


C:\Kaggle\WSDM-zh\code\iwillwin\data_utils\data_helpers.py:132: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_df = pd.concat((train_df, test_df))
C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 5196


## Load and prepare the data

In [4]:
print("Embeddings")
print(os.listdir("../data/wordvec"))

Embeddings
['.gitkeep', 'fasttext-50-win3.vec', 'zh-wordvec-50-cbow-windowsize50.vec', 'zh-wordvec-50-skipgram-windowsize7.vec']


In [5]:
data_loader = DataLoader()
skip_gram_embeddings = data_loader.load_embedding('../data/wordvec/zh-wordvec-50-skipgram-windowsize7.vec')
cbow_embeddings = data_loader.load_embedding('../data/wordvec/zh-wordvec-50-cbow-windowsize50.vec')
fasttext_embeddings = data_loader.load_embedding('../data/wordvec/fasttext-50-win3.vec')

Total 4114 word vectors.
Total 4114 word vectors.
Total 4114 word vectors.


In [6]:
def build_embedding_matrix(embeddings_index, nb_words=NB_WORDS, word_index=data_transformer.tokenizer.word_index):
    #nb_words = min(nb_words, len(embeddings_index))
    embedding_matrix = np.random.rand(nb_words, 50)
    embedding_matrix = np.zeros((nb_words, 50))
    
    word_index = data_transformer.tokenizer.word_index
    null_words = open('null-word.txt', 'w', encoding='utf-8')

    for word, i in word_index.items():
        if i >= nb_words:
            null_words.write(word + '\n')
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            null_words.write(word + '\n')
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix

In [7]:
cbow_matrix = build_embedding_matrix(cbow_embeddings)
skipgram_matrix = build_embedding_matrix(skip_gram_embeddings)
fasttext_matrix = build_embedding_matrix(fasttext_embeddings)

Null word embeddings: 948
Null word embeddings: 948
Null word embeddings: 948


In [8]:
meta_embeddings = np.concatenate((cbow_matrix, skipgram_matrix, fasttext_matrix), axis=1)

In [9]:
meta_embeddings[0] = np.array([0] * 150) # zero padding

# Add tricky features

## Rumor words

In [10]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [11]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [12]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests[2], test_has_rumor.reshape((-1, 1))), axis=1)

## Exact Match

In [13]:
def _build_exact_match_sequences(sent_1, sent_2):
    sent_1_char_set = set(sent_1)
    sent_2_char_set = set(sent_2)
    intersection = sent_1_char_set & sent_2_char_set
    
    sent_1_em = np.zeros_like(sent_1)
    sent_2_em = np.zeros_like(sent_2)

    for i in range(len(sent_1)):
        if sent_1[i] == 0:
            continue
        if sent_1[i] in intersection:
            sent_1_em[i] = 1
    
    for i in range(len(sent_2)):
        if sent_2[i] == 0:
            continue        
        if sent_2[i] in intersection:
            sent_2_em[i] = 1
    
    return sent_1_em, sent_2_em

def build_exact_match_sequences(sents_1, sents_2):
    sents_1_em, sents_2_em = [], []
    for sent_1, sent_2 in zip(sents_1, sents_2):
        sent_1_em, sent_2_em = _build_exact_match_sequences(sent_1, sent_2)
        sents_1_em.append(sent_1_em)
        sents_2_em.append(sent_2_em)
    return np.array(sents_1_em), np.array(sents_2_em)

In [14]:
%%time
trains_1_ems, trains_2_ems = build_exact_match_sequences(trains[0], trains[1])
tests_1_ems, tests_2_ems = build_exact_match_sequences(tests[0], tests[1])

Wall time: 16 s


In [15]:
print("Shape of train em", trains_1_ems.shape, trains_2_ems.shape)
print("Shape of test em", tests_1_ems.shape, tests_2_ems.shape)

Shape of train em (320552, 50) (320552, 50)
Shape of test em (80126, 50) (80126, 50)


In [16]:
em_train_features = (trains_1_ems, trains_2_ems)
em_test_features = (tests_1_ems, tests_2_ems)

In [17]:
#class_weight.compute_class_weight('balanced', [0, 1, 2], labels.argmax(axis=1))

# Load Ensemble Inputs

In [18]:
ensemble_inputs = pd.read_csv("../data/ensemble/second_level/FirstLevelPseudoLabels.csv")
pseudo_labels = ensemble_inputs[['unrelated', 'agreed', 'disagreed']].values

# Trick or Treat!!!

In [19]:
use_tricky = True

if use_tricky:
    trains = (trains[0], trains[1], trick_trains_features)
    tests = (tests[0], tests[1], trick_tests_features)
else:
    trains = (trains[0], trains[1], trains[2])
    tests = (tests[0], tests[1], tests[2])

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, X, y, fold_count, em_train_features, batch_size, get_model_func, tests, em_test_features, pseudo_labels, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        X1, X2, features, = X
        em1, em2 = em_train_features
        features = features
        #features = features[:, -1]
        weight_val=scale_sample_weight

        fold_size = len(X1) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(X1)

            train_x1 = np.concatenate([X1[:fold_start], X1[fold_end:], tests[0]])
            train_x2 = np.concatenate([X2[:fold_start], X2[fold_end:], tests[1]])
            train_features = np.concatenate([features[:fold_start], features[fold_end:], tests[2]])
            
            train_em_1 = np.concatenate([em1[:fold_start], em1[fold_end:], em_test_features[0]])
            train_em_2 = np.concatenate([em2[:fold_start], em2[fold_end:], em_test_features[1]])
            
            train_y = np.concatenate([y[:fold_start], y[fold_end:], pseudo_labels])

            val_x1 = X1[fold_start:fold_end]
            val_x2 = X2[fold_start:fold_end]
            val_features = features[fold_start:fold_end]
            val_em1 = em1[fold_start:fold_end]
            val_em2 = em2[fold_start:fold_end]
            val_y = y[fold_start:fold_end]

            fold_pos = (np.sum(train_y) / len(train_x1))

            train_data = {
                "first_sentences": train_x1,
                "second_sentences": train_x2,
                "mata-features": train_features,
                "first_exact_match": train_em_1,
                "second_exact_match": train_em_2,
            }

            val_data = {
                "first_sentences": val_x1,
                "second_sentences": val_x2,
                "mata-features": val_features,
                "first_exact_match": val_em1,
                "second_exact_match": val_em2,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_weighted_accuracy', patience=patience)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        model.load_weights(bst_model_path)
        print("Load the model from ", bst_model_path)
        fine_tune_model_path = self.model_stamp + "-fine-tune-class_scaled-" + str(fold_id) + '.h5'
        
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(fine_tune_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         callbacks=[early_stopping, model_checkpoint],
                         class_weight={0: 1/16, 1: 1/15, 2: 1/5})
        bst_val_score = max(hist.history['val_weighted_accuracy'])
        model.load_weights(fine_tune_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [21]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = tf.nn.leaky_relu(K.conv1d(u_vecs, self.W))
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = K.tanh(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)
    
def get_padding_mask(q, k):
    ones = K.expand_dims(K.ones_like(q, 'float32'), -1)
    mask = K.cast(K.expand_dims(K.not_equal(k, 0), 1), 'float32')
    mask = K.batch_dot(ones, mask, axes=[2,1])
    return mask

def unchanged_shape(input_shape):
    "Function for Lambda layer"
    return input_shape

def substract(input_1, input_2):
    "Substract element-wise"
    out_ = Lambda(lambda x: x[0] - x[1])([input_1, input_2])
    return out_

def eldistance(input_1, input_2):
    "Substract element-wise"
    out_ = Lambda(lambda x: K.sqrt(K.square(x[0] - x[1])))([input_1, input_2])
    return out_

def submult(input_1, input_2):
    "Get multiplication and subtraction then concatenate results"
    mult = Multiply()([input_1, input_2])
    add = Add()([input_1, input_2])
    sub = substract(input_1, input_2)
    distance = eldistance(input_1, input_2)
    
    dual = Concatenate()([input_1, input_2])
    dual = Dense(32, activation='relu')(dual)
    dual = Dropout(0.5)(dual)
    dual = Dense(8, activation='relu')(dual)
    
    out_= Concatenate()([sub, mult, ])
    return out_

def apply_multiple(input_, layers):
    "Apply layers to input then concatenate result"
    if not len(layers) > 1:
        raise ValueError('Layers list should contain more than 1 layer')
    else:
        agg_ = []
        for layer in layers:
            agg_.append(layer(input_))
        out_ = Concatenate()(agg_)
    return out_

def time_distributed(input_, layers):
    "Apply a list of layers in TimeDistributed mode"
    out_ = []
    node_ = input_
    for layer_ in layers:
        node_ = TimeDistributed(layer_)(node_)
    out_ = node_
    return out_

def soft_attention_alignment(input_1, input_2):
    "Align text representation with neural soft attention"
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2, 1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned    

def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

def get_dense_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q1_c = Input(shape=(max_sequence_length, 11), name='first_sentences_char')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q2_c = Input(shape=(max_sequence_length, 11), name='second_sentences_char')
    input_layer_3 = Input(shape=(len(model_config.META_FEATURES),), name='mata-features', dtype="float32")

    embedding = Embedding(nb_words,
                            embedding_dim,
                            input_length=max_sequence_length,
                            trainable=True)
    
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.5)(q1_embed)
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.5)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    
    q1_encoded = th(q1_embed,)    
    q2_encoded = th(q2_embed,)
    
    q1_aligned, q2_aligned = soft_attention_alignment(q1_encoded, q2_encoded)
    q1_encoded = Concatenate()([q2_aligned, q1_encoded])
    q2_encoded = Concatenate()([q1_aligned, q2_encoded])  
    
    cnn_init = Conv1D(32, 1, strides=1, padding='same', activation='relu')
    q1_seq = cnn_init(q1_encoded)
    q2_seq = cnn_init(q2_encoded)
    
    cnns = [Conv1D(1, 3, strides=1, padding='same', activation='relu') for i in range(10)]
    
    for idx, cnn in enumerate(cnns):
        q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
        q1_encoded = Concatenate()([q1_seq, q2_aligned, q1_encoded])
        q2_encoded = Concatenate()([q2_seq, q1_aligned, q2_encoded])            
        q1_seq = cnn(q1_encoded)
        q2_seq = cnn(q2_encoded)    
    
    
    capsule_pooling = Capsule(num_capsule=8, dim_capsule=100, routings=3, share_weights=True)
    
    # Pooling
    q1_rep = Flatten()(capsule_pooling(q1_encoded))
    q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    
    #q1_rep = apply_multiple(q1_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(),])
    #q2_rep = apply_multiple(q2_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(),])    
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi, ])
    h_all = Dropout(0.5)(h_all)
    h_all = Dense(256, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, input_layer_3], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy'])
    model.summary()
    return model

In [22]:
from keras import regularizers

def get_darnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    input_layer_3 = Input(shape=(1,), name='mata-features', dtype="float32")

    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
 
    em_embeddings = Embedding(2, 1,
                     input_length=max_sequence_length,
                     trainable=True)   
    
    q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match)])
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    
    q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match)])
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    #q1_embed = Dropout(0.2)(th(q1_embed,))
    #q2_embed = Dropout(0.2)(th(q2_embed,))    
    
    rnns = [CuDNNGRU(42, return_sequences=True) for i in range(3)]
    
    q1_res = []
    q2_res = []
    
    
    for idx, rnn in enumerate(rnns):
        q1_seq = rnn(q1_embed)
        q1_seq = Dropout(0.1)(q1_seq)
        q2_seq = rnn(q2_embed)
        q2_seq = Dropout(0.1)(q2_seq)
        q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
        
        q1_res.append(q2_aligned)
        q1_res.append(q1_seq)
        
        q2_res.append(q1_aligned)
        q2_res.append(q2_seq)
        
        q1_embed = Concatenate()([q1_seq, q2_aligned, q1_embed])
        q2_embed = Concatenate()([q2_seq, q1_aligned, q2_embed])            
        
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    q1_res = Concatenate()(q1_res)
    q2_res = Concatenate()(q2_res)
    
    q1_rep = apply_multiple(q1_res, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_res, [GlobalAvgPool1D(), GlobalMaxPool1D()])    
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.1)(h_all)
    h_all = Dense(256, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [23]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    label_weights = K.max(K.cast(weight_mask, 'float32'), axis=-1)
    
    true_label = K.argmax(y_true, axis=-1)
    pred_label = K.argmax(y_pred, axis=-1)
    
    res = K.cast(K.equal(true_label, pred_label), tf.float32) * label_weights / K.sum(label_weights)
    res = K.sum(res)
    return res

In [24]:
model_manager = ModelManager()

# DenseRNN

In [31]:
fold_count = 8
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_darnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=(trains[0], trains[1], trains[2][:, -1]), y=labels, augments=None, fold_count=fold_count, batch_size=128,
        tests=(tests[0], tests[1], tests[2][:, -1]), em_test_features=em_test_features, pseudo_labels=pseudo_labels,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=10)

    print("score", score)
    oofs_dir = "../data/pseudo/oofs/"
    output_dir = "../data/pseudo/output/"
    onehot_pred_dir = "../data/pseudo/one_hot_pred/"

    model_submit_prefix = "PS3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2][:, -1],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:200: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
first_exact_match (InputLayer)  (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
second_exact_match (InputLayer) (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM0.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 124s 344us/step - loss: 0.0204 - acc: 0.8848 - weighted_accuracy: 0.8794 - val_loss: 0.3234 - val_acc: 0.8497 - val_weighted_accuracy: 0.8439
Epoch 2/500
360609/360609 [==============================] - 122s 339us/step - loss: 0.0202 - acc: 0.8850 - weighted_accuracy: 0.8804 - val_loss: 0.3291 - val_acc: 0.8476 - val_weighted_accuracy: 0.8422
Epoch 3/500
360609/360609 [==============================] - 122s 339us/step - loss: 0.0200 - acc: 0.8864 - weighted_accuracy: 0.8820 - val_loss: 0.3216 - val_acc: 0.8506 - val_weighted_accuracy: 0.8475
Epoch 4/500
360609/360609 [==============================] - 122s 340us/step - loss: 0.0199 - acc: 0.8872 - weighted_accuracy: 0.8832 - val_loss: 0.3234 - val_acc: 0.8520 - val_weighted_accuracy: 0.8444
Epoch 5/500
360609/360609 [===================

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM1.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 124s 344us/step - loss: 0.0192 - acc: 0.8928 - weighted_accuracy: 0.8884 - val_loss: 0.2893 - val_acc: 0.8701 - val_weighted_accuracy: 0.8627
Epoch 2/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0192 - acc: 0.8926 - weighted_accuracy: 0.8886 - val_loss: 0.2918 - val_acc: 0.8679 - val_weighted_accuracy: 0.8611
Epoch 3/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0192 - acc: 0.8929 - weighted_accuracy: 0.8890 - val_loss: 0.3004 - val_acc: 0.8654 - val_weighted_accuracy: 0.8603
Epoch 4/500
360609/360609 [==============================] - 127s 351us/step - loss: 0.0192 - acc: 0.8927 - weighted_accuracy: 0.8888 - val_loss: 0.2967 - val_acc: 0.8647 - val_weighted_accuracy: 0.8591
Epoch 5/500
360609/360609 [===================

...

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM3.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 124s 345us/step - loss: 0.0195 - acc: 0.8910 - weighted_accuracy: 0.8860 - val_loss: 0.3165 - val_acc: 0.8571 - val_weighted_accuracy: 0.8538
Epoch 2/500
360609/360609 [==============================] - 123s 340us/step - loss: 0.0195 - acc: 0.8908 - weighted_accuracy: 0.8862 - val_loss: 0.3201 - val_acc: 0.8566 - val_weighted_accuracy: 0.8523
Epoch 3/500
360609/360609 [==============================] - 123s 340us/step - loss: 0.0195 - acc: 0.8913 - weighted_accuracy: 0.8866 - val_loss: 0.3135 - val_acc: 0.8594 - val_weighted_accuracy: 0.8558
Epoch 4/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0195 - acc: 0.8907 - weighted_accuracy: 0.8869 - val_loss: 0.3000 - val_acc: 0.8664 - val_weighted_accuracy: 0.8543
Epoch 5/500
360609/360609 [===================

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM4.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 125s 347us/step - loss: 0.0191 - acc: 0.8943 - weighted_accuracy: 0.8895 - val_loss: 0.3289 - val_acc: 0.8509 - val_weighted_accuracy: 0.8426
Epoch 2/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0191 - acc: 0.8936 - weighted_accuracy: 0.8891 - val_loss: 0.3294 - val_acc: 0.8482 - val_weighted_accuracy: 0.8425
Epoch 3/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0191 - acc: 0.8942 - weighted_accuracy: 0.8898 - val_loss: 0.3250 - val_acc: 0.8497 - val_weighted_accuracy: 0.8426
Epoch 4/500
360609/360609 [==============================] - 123s 340us/step - loss: 0.0192 - acc: 0.8931 - weighted_accuracy: 0.8888 - val_loss: 0.3281 - val_acc: 0.8488 - val_weighted_accuracy: 0.8422
Epoch 5/500
360609/360609 [===================

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM5.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 125s 347us/step - loss: 0.0201 - acc: 0.8872 - weighted_accuracy: 0.8821 - val_loss: 0.3539 - val_acc: 0.8292 - val_weighted_accuracy: 0.8264
Epoch 2/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0199 - acc: 0.8885 - weighted_accuracy: 0.8836 - val_loss: 0.3377 - val_acc: 0.8379 - val_weighted_accuracy: 0.8265
Epoch 3/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0198 - acc: 0.8885 - weighted_accuracy: 0.8841 - val_loss: 0.3613 - val_acc: 0.8249 - val_weighted_accuracy: 0.8198
Epoch 4/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0197 - acc: 0.8893 - weighted_accuracy: 0.8850 - val_loss: 0.3621 - val_acc: 0.8284 - val_weighted_accuracy: 0.8237
Epoch 5/500
360609/360609 [===================

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM6.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 126s 348us/step - loss: 0.0203 - acc: 0.8851 - weighted_accuracy: 0.8800 - val_loss: 0.3043 - val_acc: 0.8627 - val_weighted_accuracy: 0.8573
Epoch 2/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0202 - acc: 0.8851 - weighted_accuracy: 0.8807 - val_loss: 0.3148 - val_acc: 0.8562 - val_weighted_accuracy: 0.8536
Epoch 3/500
360609/360609 [==============================] - 123s 342us/step - loss: 0.0201 - acc: 0.8866 - weighted_accuracy: 0.8821 - val_loss: 0.3187 - val_acc: 0.8503 - val_weighted_accuracy: 0.8533
Epoch 4/500
360609/360609 [==============================] - 123s 342us/step - loss: 0.0200 - acc: 0.8874 - weighted_accuracy: 0.8830 - val_loss: 0.3098 - val_acc: 0.8595 - val_weighted_accuracy: 0.8561
Epoch 5/500
360609/360609 [===================

Load the model from  3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM7.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 126s 351us/step - loss: 0.0195 - acc: 0.8909 - weighted_accuracy: 0.8861 - val_loss: 0.3303 - val_acc: 0.8522 - val_weighted_accuracy: 0.8505
Epoch 2/500
360609/360609 [==============================] - 124s 344us/step - loss: 0.0194 - acc: 0.8910 - weighted_accuracy: 0.8864 - val_loss: 0.2965 - val_acc: 0.8656 - val_weighted_accuracy: 0.8572
Epoch 3/500
360609/360609 [==============================] - 123s 342us/step - loss: 0.0194 - acc: 0.8918 - weighted_accuracy: 0.8874 - val_loss: 0.2967 - val_acc: 0.8634 - val_weighted_accuracy: 0.8568
Epoch 4/500
360609/360609 [==============================] - 123s 342us/step - loss: 0.0194 - acc: 0.8914 - weighted_accuracy: 0.8876 - val_loss: 0.3014 - val_acc: 0.8650 - val_weighted_accuracy: 0.8583
Epoch 5/500
360609/360609 [===================

# DenseCNN

In [32]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

def get_dense_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    meta_features_input = Input(shape=(1,), name='mata-features')
    
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
    
    flex_embedding = Embedding(nb_words, 20,
                      input_length=max_sequence_length,
                      trainable=True)
    
    em_embeddings = Reshape((max_sequence_length, 1))
    
    q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match),])
    q1_encoded = SpatialDropout1D(0.2)(q1_embed)
    
    q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match),])
    q2_encoded = SpatialDropout1D(0.2)(q2_embed)


    #capsule_pooling = Capsule(num_capsule=3, dim_capsule=600, routings=2, share_weights=True)
    nb_filters = 64
    
    cnns = [Conv1D(64, 1, strides=1, padding='same', activation='relu') for i in range(3)]
    gates_cnns = [Conv1D(nb_filters, 3, dilation_rate=1, padding='same', activation='tanh') for i in range(3)]
    sigm_cnns = [Conv1D(nb_filters, 3, dilation_rate=1, padding='same', activation='sigmoid') for i in range(3)]
    
    for i in range(len(cnns)):
        drop = Dropout(0.1)
        q1_t = gates_cnns[i](q1_encoded)
        q2_t = gates_cnns[i](q2_encoded)    
        
        q1_s = sigm_cnns[i](q1_encoded)
        q2_s = sigm_cnns[i](q2_encoded)        
        
        q1_x = Multiply()([q1_t, q1_s])
        q1_x = cnns[i](q1_x)
        q1_x = drop(q1_x)
        
        q2_x = Multiply()([q2_t, q2_s])
        q2_x = cnns[i](q2_x)
        q2_x = drop(q2_x)

        q1_aligned, q2_aligned = soft_attention_alignment(q1_x, q2_x)
        q1_encoded = Concatenate()([q1_x, q2_aligned, q1_encoded])
        q2_encoded = Concatenate()([q2_x, q1_aligned, q2_encoded]) 
    
    #capsule_pooling = Capsule(num_capsule=3, dim_capsule=600, routings=2, share_weights=True)
    
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    attn = AttentionWeightedAverage()
    
    
    q1_rep = apply_multiple(q1_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])
    q2_rep = apply_multiple(q2_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])    
    
    
    #meta_features = BatchNormalization()(meta_features_input)
    #meta_features = Dropout(0.8)(meta_features)
    #meta_features = Highway(activation='relu')(meta_features)
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    #drops = Dropout(0.3)
    #q1_rep = shrink(drops(q1_rep))
    #q2_rep = shrink(drops(q2_rep))
    #meta_features = BatchNormalization()(meta_features_input)
    #meta_features = Dropout(0.8)(meta_features)
    #meta_features = Highway(activation='relu')(meta_features)
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.2)(h_all)
    h_all = BatchNormalization()(h_all)
    #h_all = Dropout(0.35)(h_all)
    #h_all = Highway(activation='relu')(h_all)
    #h_all = Highway(activation='relu')(h_all)    
    h_all = Dense(64, activation='relu', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(1e-6))(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, meta_features_input, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [33]:
fold_count = 8
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=(trains[0], trains[1], trains[2][:, -1]), y=labels, augments=None, fold_count=fold_count, batch_size=128,
        tests=(tests[0], tests[1], tests[2][:, -1]), em_test_features=em_test_features, pseudo_labels=pseudo_labels,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=10)

    print("score", score)
    oofs_dir = "../data/pseudo/oofs/"
    output_dir = "../data/pseudo/output/"
    onehot_pred_dir = "../data/pseudo/one_hot_pred/"

    model_submit_prefix = "PS3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2][:, -1],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
first_exact_match (InputLayer)  (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
second_exact_match (InputLayer) (None, 50)           0                                            
_____________________________________________________________________________________________

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM0.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 123s 342us/step - loss: 0.0235 - acc: 0.8662 - weighted_accuracy: 0.8600 - val_loss: 0.3166 - val_acc: 0.8556 - val_weighted_accuracy: 0.8442
Epoch 2/500
360609/360609 [==============================] - 119s 331us/step - loss: 0.0235 - acc: 0.8683 - weighted_accuracy: 0.8625 - val_loss: 0.3138 - val_acc: 0.8566 - val_weighted_accuracy: 0.8431
Epoch 3/500
360609/360609 [==============================] - 119s 331us/step - loss: 0.0233 - acc: 0.8708 - weighted_accuracy: 0.8652 - val_loss: 0.3116 - val_acc: 0.8566 - val_weighted_accuracy: 0.8427
Epoch 4/500
360609/360609 [==============================] - 119s 331us/step - loss: 0.0232 - acc: 0.8726 - weighted_accuracy: 0.8672 - val_loss: 0.3133 - val_acc: 0.8586 - val_weighted_accuracy: 0.8460
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM1.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 123s 341us/step - loss: 0.0227 - acc: 0.8778 - weighted_accuracy: 0.8722 - val_loss: 0.2989 - val_acc: 0.8673 - val_weighted_accuracy: 0.8570
Epoch 2/500
360609/360609 [==============================] - 120s 331us/step - loss: 0.0230 - acc: 0.8757 - weighted_accuracy: 0.8702 - val_loss: 0.3049 - val_acc: 0.8631 - val_weighted_accuracy: 0.8553
Epoch 3/500
360609/360609 [==============================] - 120s 331us/step - loss: 0.0229 - acc: 0.8763 - weighted_accuracy: 0.8713 - val_loss: 0.3066 - val_acc: 0.8612 - val_weighted_accuracy: 0.8535
Epoch 4/500
360609/360609 [==============================] - 120s 332us/step - loss: 0.0230 - acc: 0.8760 - weighted_accuracy: 0.8706 - val_loss: 0.2850 - val_acc: 0.8735 - val_weighted_accuracy: 0.8572
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM2.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 123s 342us/step - loss: 0.0231 - acc: 0.8714 - weighted_accuracy: 0.8650 - val_loss: 0.3253 - val_acc: 0.8493 - val_weighted_accuracy: 0.8436
Epoch 2/500
360609/360609 [==============================] - 120s 332us/step - loss: 0.0233 - acc: 0.8707 - weighted_accuracy: 0.8644 - val_loss: 0.3288 - val_acc: 0.8490 - val_weighted_accuracy: 0.8448
Epoch 3/500
360609/360609 [==============================] - 120s 332us/step - loss: 0.0232 - acc: 0.8733 - weighted_accuracy: 0.8674 - val_loss: 0.3208 - val_acc: 0.8524 - val_weighted_accuracy: 0.8394
Epoch 4/500
360609/360609 [==============================] - 120s 332us/step - loss: 0.0230 - acc: 0.8744 - weighted_accuracy: 0.8687 - val_loss: 0.3213 - val_acc: 0.8530 - val_weighted_accuracy: 0.8452
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM3.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 124s 344us/step - loss: 0.0229 - acc: 0.8751 - weighted_accuracy: 0.8690 - val_loss: 0.3194 - val_acc: 0.8581 - val_weighted_accuracy: 0.8498
Epoch 2/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0231 - acc: 0.8740 - weighted_accuracy: 0.8679 - val_loss: 0.3107 - val_acc: 0.8614 - val_weighted_accuracy: 0.8479
Epoch 3/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0231 - acc: 0.8750 - weighted_accuracy: 0.8690 - val_loss: 0.3208 - val_acc: 0.8567 - val_weighted_accuracy: 0.8501
Epoch 4/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0230 - acc: 0.8764 - weighted_accuracy: 0.8705 - val_loss: 0.3017 - val_acc: 0.8650 - val_weighted_accuracy: 0.8481
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM4.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 128s 354us/step - loss: 0.0225 - acc: 0.8795 - weighted_accuracy: 0.8740 - val_loss: 0.3434 - val_acc: 0.8414 - val_weighted_accuracy: 0.8342
Epoch 2/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0229 - acc: 0.8771 - weighted_accuracy: 0.8712 - val_loss: 0.3317 - val_acc: 0.8468 - val_weighted_accuracy: 0.8338
Epoch 3/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0229 - acc: 0.8785 - weighted_accuracy: 0.8727 - val_loss: 0.3181 - val_acc: 0.8542 - val_weighted_accuracy: 0.8382
Epoch 4/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0228 - acc: 0.8780 - weighted_accuracy: 0.8719 - val_loss: 0.3190 - val_acc: 0.8530 - val_weighted_accuracy: 0.8404
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM5.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 124s 345us/step - loss: 0.0228 - acc: 0.8744 - weighted_accuracy: 0.8680 - val_loss: 0.3337 - val_acc: 0.8464 - val_weighted_accuracy: 0.8308
Epoch 2/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0231 - acc: 0.8736 - weighted_accuracy: 0.8673 - val_loss: 0.3431 - val_acc: 0.8391 - val_weighted_accuracy: 0.8237
Epoch 3/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0230 - acc: 0.8749 - weighted_accuracy: 0.8691 - val_loss: 0.3404 - val_acc: 0.8409 - val_weighted_accuracy: 0.8295
Epoch 4/500
360609/360609 [==============================] - 120s 333us/step - loss: 0.0229 - acc: 0.8760 - weighted_accuracy: 0.8705 - val_loss: 0.3414 - val_acc: 0.8406 - val_weighted_accuracy: 0.8294
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM6.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 125s 346us/step - loss: 0.0225 - acc: 0.8817 - weighted_accuracy: 0.8762 - val_loss: 0.3072 - val_acc: 0.8627 - val_weighted_accuracy: 0.8537
Epoch 2/500
360609/360609 [==============================] - 120s 334us/step - loss: 0.0229 - acc: 0.8786 - weighted_accuracy: 0.8730 - val_loss: 0.3076 - val_acc: 0.8619 - val_weighted_accuracy: 0.8507
Epoch 3/500
360609/360609 [==============================] - 120s 334us/step - loss: 0.0228 - acc: 0.8787 - weighted_accuracy: 0.8732 - val_loss: 0.3014 - val_acc: 0.8633 - val_weighted_accuracy: 0.8536
Epoch 4/500
360609/360609 [==============================] - 120s 334us/step - loss: 0.0227 - acc: 0.8795 - weighted_accuracy: 0.8742 - val_loss: 0.3046 - val_acc: 0.8634 - val_weighted_accuracy: 0.8547
Epoch 5/500
360609/360609 [==========================

Load the model from  3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM7.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 125s 347us/step - loss: 0.0232 - acc: 0.8705 - weighted_accuracy: 0.8644 - val_loss: 0.2997 - val_acc: 0.8632 - val_weighted_accuracy: 0.8499
Epoch 2/500
360609/360609 [==============================] - 121s 334us/step - loss: 0.0235 - acc: 0.8705 - weighted_accuracy: 0.8647 - val_loss: 0.2967 - val_acc: 0.8636 - val_weighted_accuracy: 0.8506
Epoch 3/500
360609/360609 [==============================] - 121s 334us/step - loss: 0.0234 - acc: 0.8719 - weighted_accuracy: 0.8663 - val_loss: 0.2947 - val_acc: 0.8674 - val_weighted_accuracy: 0.8550
Epoch 4/500
360609/360609 [==============================] - 121s 334us/step - loss: 0.0232 - acc: 0.8735 - weighted_accuracy: 0.8684 - val_loss: 0.2924 - val_acc: 0.8672 - val_weighted_accuracy: 0.8511
Epoch 5/500
360609/360609 [==========================

# ESIM

In [25]:
from keras import regularizers

def get_ESIM(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")

    #input_encoded = BatchNormalization()(input_layer_3)
    input_encoded = Dense(2016, activation='elu')(input_layer_3)
    input_encoded = Dropout(0.25)(input_encoded)
    
    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
 
    em_embeddings = Embedding(2, 1,
                     input_length=max_sequence_length,
                     trainable=True)   
    
    #q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match)])
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    
    #q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match)])
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    batch_norm = BatchNormalization(axis=-1)
    q1_embed = batch_norm(q1_embed)
    q2_embed = batch_norm(q2_embed)
    
    aggreation_gru = Bidirectional(CuDNNLSTM(72, return_sequences=True))
 
    q1_seq = aggreation_gru(q1_embed)
    q2_seq = aggreation_gru(q2_embed)
        
    q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
    q1_vec = Concatenate()([q1_seq, q2_aligned, substract(q1_seq, q2_aligned), Multiply()([q1_seq, q2_aligned])])
    q2_vec = Concatenate()([q2_seq, q1_aligned, substract(q2_seq, q1_aligned), Multiply()([q2_seq, q1_aligned])])
    
    compare_gru = Bidirectional(CuDNNLSTM(72, return_sequences=True))
    
    q1_rep = compare_gru(q1_vec)
    q2_rep = compare_gru(q2_vec)
    
    q1_rep = apply_multiple(q1_rep, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_rep, [GlobalAvgPool1D(), GlobalMaxPool1D()])    
    
    h_all = Concatenate()([q1_rep, q2_rep])
    h_all = BatchNormalization()(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = BatchNormalization()(h_all)
    h_all = Dropout(0.2)(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = BatchNormalization()(h_all)
    h_all = Dropout(0.2)(h_all)
    
    out_ = Dense(3, activation='softmax')(h_all)
    
    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5, amsgrad=True), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [26]:
fold_count = 8
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_ESIM(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []

    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=(trains[0], trains[1], trains[2]), y=labels, augments=None, fold_count=fold_count, batch_size=128,
        tests=(tests[0], tests[1], tests[2]), em_test_features=em_test_features, pseudo_labels=pseudo_labels,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=10)

    print("score", score)
    oofs_dir = "../data/pseudo/oofs/"
    output_dir = "../data/pseudo/output/"
    onehot_pred_dir = "../data/pseudo/one_hot_pred/"

    model_submit_prefix = "PS3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 150)      750000      first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 50, 150)      0           embedding_1[0][0]           

Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 175s 487us/step - loss: 0.0200 - acc: 0.8897 - weighted_accuracy: 0.8847 - val_loss: 0.3073 - val_acc: 0.8615 - val_weighted_accuracy: 0.8539
Epoch 2/500
360609/360609 [==============================] - 157s 435us/step - loss: 0.0187 - acc: 0.8978 - weighted_accuracy: 0.8939 - val_loss: 0.2906 - val_acc: 0.8703 - val_weighted_accuracy: 0.8590
Epoch 3/500
360609/360609 [==============================] - 155s 431us/step - loss: 0.0178 - acc: 0.9052 - weighted_accuracy: 0.9022 - val_loss: 0.3017 - val_acc: 0.8665 - val_weighted_accuracy: 0.8598
Epoch 4/500
360609/360609 [==============================] - 157s 435us/step - loss: 0.0171 - acc: 0.9093 - weighted_accuracy: 0.9071 - val_loss: 0.2890 - val_acc: 0.8714 - val_weighted_accuracy: 0.8603
Epoch 5/500
360609/360609 [==============================] - 155s 431us/step - loss: 0.0164 - acc: 0.9133 - weighted_accuracy: 0.9116 - v

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM1.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 157s 437us/step - loss: 0.0186 - acc: 0.9001 - weighted_accuracy: 0.8957 - val_loss: 0.3110 - val_acc: 0.8702 - val_weighted_accuracy: 0.8660
Epoch 2/500
360609/360609 [==============================] - 155s 430us/step - loss: 0.0177 - acc: 0.9058 - weighted_accuracy: 0.9028 - val_loss: 0.2828 - val_acc: 0.8757 - val_weighted_accuracy: 0.8686
Epoch 3/500
360609/360609 [==============================] - 156s 434us/step - loss: 0.0170 - acc: 0.9105 - weighted_accuracy: 0.9080 - val_loss: 0.2793 - val_acc: 0.8770 - val_weighted_accuracy: 0.8700
Epoch 4/500
360609/360609 [==============================] - 155s 430us/step - loss: 0.0164 - acc: 0.9144 - weighted_accuracy: 0.9127 - val_loss: 0.2729 - val_acc: 0.8811 - val_weighted_accuracy: 0.8739
Epoch 5/500
360609/360609 [==============================] - 

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM2.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 171s 474us/step - loss: 0.0192 - acc: 0.8959 - weighted_accuracy: 0.8911 - val_loss: 0.3029 - val_acc: 0.8640 - val_weighted_accuracy: 0.8577
Epoch 2/500
360609/360609 [==============================] - 168s 466us/step - loss: 0.0182 - acc: 0.9025 - weighted_accuracy: 0.8987 - val_loss: 0.2952 - val_acc: 0.8680 - val_weighted_accuracy: 0.8605
Epoch 3/500
360609/360609 [==============================] - 168s 467us/step - loss: 0.0174 - acc: 0.9082 - weighted_accuracy: 0.9053 - val_loss: 0.3029 - val_acc: 0.8659 - val_weighted_accuracy: 0.8595
Epoch 4/500
360609/360609 [==============================] - 168s 467us/step - loss: 0.0168 - acc: 0.9114 - weighted_accuracy: 0.9093 - val_loss: 0.3089 - val_acc: 0.8610 - val_weighted_accuracy: 0.8577
Epoch 5/500
360609/360609 [==============================] - 

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM3.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 159s 441us/step - loss: 0.0175 - acc: 0.9078 - weighted_accuracy: 0.9041 - val_loss: 0.3190 - val_acc: 0.8588 - val_weighted_accuracy: 0.8566
Epoch 2/500
360609/360609 [==============================] - 156s 433us/step - loss: 0.0169 - acc: 0.9113 - weighted_accuracy: 0.9083 - val_loss: 0.2914 - val_acc: 0.8715 - val_weighted_accuracy: 0.8636
Epoch 3/500
360609/360609 [==============================] - 156s 433us/step - loss: 0.0162 - acc: 0.9157 - weighted_accuracy: 0.9138 - val_loss: 0.2932 - val_acc: 0.8700 - val_weighted_accuracy: 0.8625
Epoch 4/500
360609/360609 [==============================] - 156s 433us/step - loss: 0.0157 - acc: 0.9189 - weighted_accuracy: 0.9175 - val_loss: 0.2904 - val_acc: 0.8719 - val_weighted_accuracy: 0.8650
Epoch 5/500
360609/360609 [==============================] - 

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM4.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 159s 442us/step - loss: 0.0184 - acc: 0.9013 - weighted_accuracy: 0.8969 - val_loss: 0.3105 - val_acc: 0.8604 - val_weighted_accuracy: 0.8503
Epoch 2/500
360609/360609 [==============================] - 161s 445us/step - loss: 0.0176 - acc: 0.9063 - weighted_accuracy: 0.9028 - val_loss: 0.3356 - val_acc: 0.8510 - val_weighted_accuracy: 0.8465
Epoch 3/500
360609/360609 [==============================] - 158s 439us/step - loss: 0.0169 - acc: 0.9110 - weighted_accuracy: 0.9083 - val_loss: 0.3184 - val_acc: 0.8603 - val_weighted_accuracy: 0.8547
Epoch 4/500
360609/360609 [==============================] - 164s 454us/step - loss: 0.0163 - acc: 0.9151 - weighted_accuracy: 0.9132 - val_loss: 0.3074 - val_acc: 0.8632 - val_weighted_accuracy: 0.8554
Epoch 5/500
360609/360609 [==============================] - 

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM5.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 161s 446us/step - loss: 0.0184 - acc: 0.9017 - weighted_accuracy: 0.8974 - val_loss: 0.3333 - val_acc: 0.8481 - val_weighted_accuracy: 0.8425
Epoch 2/500
360609/360609 [==============================] - 157s 435us/step - loss: 0.0175 - acc: 0.9071 - weighted_accuracy: 0.9038 - val_loss: 0.3270 - val_acc: 0.8515 - val_weighted_accuracy: 0.8453
Epoch 3/500
360609/360609 [==============================] - 156s 434us/step - loss: 0.0168 - acc: 0.9110 - weighted_accuracy: 0.9084 - val_loss: 0.3239 - val_acc: 0.8536 - val_weighted_accuracy: 0.8440
Epoch 4/500
360609/360609 [==============================] - 157s 434us/step - loss: 0.0162 - acc: 0.9158 - weighted_accuracy: 0.9141 - val_loss: 0.3234 - val_acc: 0.8558 - val_weighted_accuracy: 0.8412
Epoch 5/500
360609/360609 [==============================] - 

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM6.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 160s 444us/step - loss: 0.0192 - acc: 0.8957 - weighted_accuracy: 0.8909 - val_loss: 0.2831 - val_acc: 0.8737 - val_weighted_accuracy: 0.8576
Epoch 2/500
360609/360609 [==============================] - 156s 433us/step - loss: 0.0182 - acc: 0.9020 - weighted_accuracy: 0.8983 - val_loss: 0.3233 - val_acc: 0.8553 - val_weighted_accuracy: 0.8561
Epoch 3/500
360609/360609 [==============================] - 157s 434us/step - loss: 0.0174 - acc: 0.9079 - weighted_accuracy: 0.9048 - val_loss: 0.2897 - val_acc: 0.8695 - val_weighted_accuracy: 0.8628
Epoch 4/500
360609/360609 [==============================] - 157s 436us/step - loss: 0.0167 - acc: 0.9123 - weighted_accuracy: 0.9101 - val_loss: 0.2807 - val_acc: 0.8746 - val_weighted_accuracy: 0.8666
Epoch 5/500
360609/360609 [==============================] - 

Load the model from  3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM7.h5
Train on 360609 samples, validate on 40069 samples
Epoch 1/500
360609/360609 [==============================] - 163s 453us/step - loss: 0.0180 - acc: 0.9050 - weighted_accuracy: 0.9011 - val_loss: 0.2756 - val_acc: 0.8788 - val_weighted_accuracy: 0.8632
Epoch 2/500
360609/360609 [==============================] - 159s 440us/step - loss: 0.0172 - acc: 0.9090 - weighted_accuracy: 0.9063 - val_loss: 0.2821 - val_acc: 0.8750 - val_weighted_accuracy: 0.8649
Epoch 3/500
360609/360609 [==============================] - 158s 437us/step - loss: 0.0165 - acc: 0.9137 - weighted_accuracy: 0.9116 - val_loss: 0.2834 - val_acc: 0.8767 - val_weighted_accuracy: 0.8669
Epoch 4/500
360609/360609 [==============================] - 159s 442us/step - loss: 0.0160 - acc: 0.9171 - weighted_accuracy: 0.9157 - val_loss: 0.2927 - val_acc: 0.8708 - val_weighted_accuracy: 0.8626
Epoch 5/500
360609/360609 [==============================] - 

# New Models

In [27]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, X, y, fold_count, em_train_features, batch_size, get_model_func, tests, em_test_features, pseudo_labels, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        X1, X2, features, = X
        em1, em2 = em_train_features
        features = features
        #features = features[:, -1]
        weight_val=scale_sample_weight

        fold_size = len(X1) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(X1)

            train_x1 = np.concatenate([X1[:fold_start], X1[fold_end:], tests[0]])
            train_x2 = np.concatenate([X2[:fold_start], X2[fold_end:], tests[1]])
            train_features = np.concatenate([features[:fold_start], features[fold_end:], tests[2]])
            
            train_em_1 = np.concatenate([em1[:fold_start], em1[fold_end:], em_test_features[0]])
            train_em_2 = np.concatenate([em2[:fold_start], em2[fold_end:], em_test_features[1]])
            
            train_y = np.concatenate([y[:fold_start], y[fold_end:], pseudo_labels])

            val_x1 = X1[fold_start:fold_end]
            val_x2 = X2[fold_start:fold_end]
            val_features = features[fold_start:fold_end]
            val_em1 = em1[fold_start:fold_end]
            val_em2 = em2[fold_start:fold_end]
            val_y = y[fold_start:fold_end]

            fold_pos = (np.sum(train_y) / len(train_x1))

            train_data = {
                "first_sentences": train_x1,
                "second_sentences": train_x2,
                "mata-features": train_features,
                "first_exact_match": train_em_1,
                "second_exact_match": train_em_2,
            }

            val_data = {
                "first_sentences": val_x1,
                "second_sentences": val_x2,
                "mata-features": val_features,
                "first_exact_match": val_em1,
                "second_exact_match": val_em2,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_weighted_accuracy', patience=patience)
        fine_tune_model_path = self.model_stamp + "-fine-tune-class_scaled-" + str(fold_id) + '.h5'
        
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(fine_tune_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         callbacks=[early_stopping, model_checkpoint],
                         class_weight={0: 1/16, 1: 1/15, 2: 1/5})
        bst_val_score = max(hist.history['val_weighted_accuracy'])
        model.load_weights(fine_tune_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [39]:
def get_decomposable_attention(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")

    #input_encoded = BatchNormalization()(input_layer_3)
    input_encoded = Dense(2016, activation='elu')(input_layer_3)
    input_encoded = Dropout(0.25)(input_encoded)
    
    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
 
    em_embeddings = Embedding(2, 1,
                     input_length=max_sequence_length,
                     trainable=True)   
    
    #q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match)])
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    
    #q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match)])
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    q1_embed = th(q1_embed)
    q2_embed = th(q2_embed)
        
    q1_aligned, q2_aligned = soft_attention_alignment(q1_embed, q2_embed)
    q1_vec = Concatenate()([q1_embed, q2_aligned, substract(q1_embed, q2_aligned), Multiply()([q1_embed, q2_aligned])])
    q2_vec = Concatenate()([q2_embed, q1_aligned, substract(q2_embed, q1_aligned), Multiply()([q2_embed, q1_aligned])])
    
    dense_compares = [
        Dense(300, activation='elu'),
        Dropout(0.2),
        Dense(200, activation='elu'),
        Dropout(0.2),
    ]

    q1_compared = time_distributed(q1_vec, dense_compares)
    q2_compared = time_distributed(q2_vec, dense_compares)
    
    q1_rep = apply_multiple(q1_compared, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_compared, [GlobalAvgPool1D(), GlobalMaxPool1D()])    
    
    h_all = Concatenate()([q1_rep, q2_rep])
    h_all = BatchNormalization()(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = Dropout(0.2)(h_all)
    h_all = BatchNormalization()(h_all)

    h_all = Dense(256, activation='elu')(h_all)
    h_all = Dropout(0.2)(h_all)
    h_all = BatchNormalization()(h_all)    
    
    out_ = Dense(3, activation='softmax')(h_all)
    
    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5, amsgrad=True), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [40]:
fold_count = 9
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-DecomposalbeAttention-NoMeta-ClassWeighted-NoEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_decomposable_attention(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=(trains[0], trains[1], trains[2]), y=labels, augments=None, fold_count=fold_count, batch_size=128,
        tests=(tests[0], tests[1], tests[2]), em_test_features=em_test_features, pseudo_labels=pseudo_labels,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=15)

    print("score", score)
    oofs_dir = "../data/pseudo/oofs/"
    output_dir = "../data/pseudo/output/"
    onehot_pred_dir = "../data/pseudo/one_hot_pred/"

    model_submit_prefix = "3Embedding-DecomposalbeAttention-NoMeta-ClassWeighted-NoEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:200: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_25 (Embedding)        (None, 50, 150)      750000      first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_25 (SpatialDr (None, 50, 150)      0           embedding_25[0][0]               
__________

Train on 365062 samples, validate on 35616 samples
Epoch 1/500
365062/365062 [==============================] - 84s 230us/step - loss: 0.0279 - acc: 0.8275 - weighted_accuracy: 0.8190 - val_loss: 0.3575 - val_acc: 0.8340 - val_weighted_accuracy: 0.8205
Epoch 2/500
365062/365062 [==============================] - 78s 214us/step - loss: 0.0238 - acc: 0.8556 - weighted_accuracy: 0.8488 - val_loss: 0.3570 - val_acc: 0.8318 - val_weighted_accuracy: 0.8277
Epoch 3/500
365062/365062 [==============================] - 78s 214us/step - loss: 0.0230 - acc: 0.8622 - weighted_accuracy: 0.8562 - val_loss: 0.3527 - val_acc: 0.8361 - val_weighted_accuracy: 0.8268
Epoch 4/500
365062/365062 [==============================] - 78s 214us/step - loss: 0.0223 - acc: 0.8676 - weighted_accuracy: 0.8622 - val_loss: 0.3259 - val_acc: 0.8510 - val_weighted_accuracy: 0.8406
Epoch 5/500
365062/365062 [==============================] - 78s 214us/step - loss: 0.0219 - acc: 0.8701 - weighted_accuracy: 0.8648 - val_lo

Train on 365062 samples, validate on 35616 samples
Epoch 1/500
365062/365062 [==============================] - 82s 225us/step - loss: 0.0283 - acc: 0.8244 - weighted_accuracy: 0.8154 - val_loss: 0.3266 - val_acc: 0.8483 - val_weighted_accuracy: 0.8422
Epoch 2/500
365062/365062 [==============================] - 79s 215us/step - loss: 0.0241 - acc: 0.8527 - weighted_accuracy: 0.8456 - val_loss: 0.3093 - val_acc: 0.8570 - val_weighted_accuracy: 0.8492
Epoch 3/500
365062/365062 [==============================] - 79s 215us/step - loss: 0.0232 - acc: 0.8599 - weighted_accuracy: 0.8536 - val_loss: 0.2987 - val_acc: 0.8619 - val_weighted_accuracy: 0.8541
Epoch 4/500
365062/365062 [==============================] - 78s 215us/step - loss: 0.0227 - acc: 0.8643 - weighted_accuracy: 0.8586 - val_loss: 0.2879 - val_acc: 0.8693 - val_weighted_accuracy: 0.8591
Epoch 5/500
365062/365062 [==============================] - 78s 215us/step - loss: 0.0222 - acc: 0.8685 - weighted_accuracy: 0.8629 - val_lo

...

Train on 365062 samples, validate on 35616 samples
Epoch 1/500
365062/365062 [==============================] - 84s 229us/step - loss: 0.0281 - acc: 0.8277 - weighted_accuracy: 0.8186 - val_loss: 0.3466 - val_acc: 0.8335 - val_weighted_accuracy: 0.8277
Epoch 2/500
365062/365062 [==============================] - 79s 217us/step - loss: 0.0238 - acc: 0.8557 - weighted_accuracy: 0.8490 - val_loss: 0.3314 - val_acc: 0.8420 - val_weighted_accuracy: 0.8327
Epoch 3/500
365062/365062 [==============================] - 79s 217us/step - loss: 0.0229 - acc: 0.8626 - weighted_accuracy: 0.8564 - val_loss: 0.3306 - val_acc: 0.8453 - val_weighted_accuracy: 0.8380
Epoch 4/500
365062/365062 [==============================] - 79s 217us/step - loss: 0.0223 - acc: 0.8677 - weighted_accuracy: 0.8619 - val_loss: 0.3121 - val_acc: 0.8539 - val_weighted_accuracy: 0.8400
Epoch 5/500
365062/365062 [==============================] - 79s 217us/step - loss: 0.0219 - acc: 0.8705 - weighted_accuracy: 0.8653 - val_lo

Train on 365062 samples, validate on 35616 samples
Epoch 1/500
365062/365062 [==============================] - 84s 230us/step - loss: 0.0278 - acc: 0.8290 - weighted_accuracy: 0.8201 - val_loss: 0.3670 - val_acc: 0.8237 - val_weighted_accuracy: 0.8181
Epoch 2/500
365062/365062 [==============================] - 80s 218us/step - loss: 0.0238 - acc: 0.8554 - weighted_accuracy: 0.8485 - val_loss: 0.3475 - val_acc: 0.8348 - val_weighted_accuracy: 0.8262
Epoch 3/500
365062/365062 [==============================] - 80s 218us/step - loss: 0.0228 - acc: 0.8632 - weighted_accuracy: 0.8569 - val_loss: 0.3513 - val_acc: 0.8317 - val_weighted_accuracy: 0.8258
Epoch 4/500
365062/365062 [==============================] - 81s 221us/step - loss: 0.0223 - acc: 0.8672 - weighted_accuracy: 0.8617 - val_loss: 0.3442 - val_acc: 0.8391 - val_weighted_accuracy: 0.8323
Epoch 5/500
365062/365062 [==============================] - 87s 239us/step - loss: 0.0218 - acc: 0.8715 - weighted_accuracy: 0.8660 - val_lo

Train on 365062 samples, validate on 35616 samples
Epoch 1/500
365062/365062 [==============================] - 84s 230us/step - loss: 0.0279 - acc: 0.8289 - weighted_accuracy: 0.8195 - val_loss: 0.3788 - val_acc: 0.8208 - val_weighted_accuracy: 0.8073
Epoch 2/500
365062/365062 [==============================] - 80s 218us/step - loss: 0.0237 - acc: 0.8561 - weighted_accuracy: 0.8493 - val_loss: 0.3625 - val_acc: 0.8225 - val_weighted_accuracy: 0.8123
Epoch 3/500
365062/365062 [==============================] - 80s 218us/step - loss: 0.0229 - acc: 0.8627 - weighted_accuracy: 0.8566 - val_loss: 0.3600 - val_acc: 0.8279 - val_weighted_accuracy: 0.8191
Epoch 4/500
365062/365062 [==============================] - 80s 218us/step - loss: 0.0223 - acc: 0.8680 - weighted_accuracy: 0.8622 - val_loss: 0.3449 - val_acc: 0.8358 - val_weighted_accuracy: 0.8233
Epoch 5/500
365062/365062 [==============================] - 84s 231us/step - loss: 0.0219 - acc: 0.8709 - weighted_accuracy: 0.8656 - val_lo

Train on 365062 samples, validate on 35616 samples
Epoch 1/500
365062/365062 [==============================] - 85s 232us/step - loss: 0.0279 - acc: 0.8283 - weighted_accuracy: 0.8190 - val_loss: 0.3614 - val_acc: 0.8254 - val_weighted_accuracy: 0.8259
Epoch 2/500
365062/365062 [==============================] - 80s 219us/step - loss: 0.0238 - acc: 0.8546 - weighted_accuracy: 0.8474 - val_loss: 0.3373 - val_acc: 0.8432 - val_weighted_accuracy: 0.8386
Epoch 3/500
365062/365062 [==============================] - 80s 219us/step - loss: 0.0230 - acc: 0.8617 - weighted_accuracy: 0.8551 - val_loss: 0.3670 - val_acc: 0.8247 - val_weighted_accuracy: 0.8289
Epoch 4/500
365062/365062 [==============================] - 80s 219us/step - loss: 0.0225 - acc: 0.8661 - weighted_accuracy: 0.8599 - val_loss: 0.3512 - val_acc: 0.8345 - val_weighted_accuracy: 0.8370
Epoch 5/500
365062/365062 [==============================] - 80s 219us/step - loss: 0.0220 - acc: 0.8699 - weighted_accuracy: 0.8641 - val_lo

Train on 365054 samples, validate on 35624 samples
Epoch 1/500
365054/365054 [==============================] - 90s 247us/step - loss: 0.0282 - acc: 0.8261 - weighted_accuracy: 0.8174 - val_loss: 0.3971 - val_acc: 0.8074 - val_weighted_accuracy: 0.8102
Epoch 2/500
365054/365054 [==============================] - 85s 233us/step - loss: 0.0240 - acc: 0.8538 - weighted_accuracy: 0.8471 - val_loss: 0.3328 - val_acc: 0.8410 - val_weighted_accuracy: 0.8371
Epoch 3/500
365054/365054 [==============================] - 85s 234us/step - loss: 0.0231 - acc: 0.8614 - weighted_accuracy: 0.8552 - val_loss: 0.3466 - val_acc: 0.8364 - val_weighted_accuracy: 0.8372
Epoch 4/500
365054/365054 [==============================] - 85s 233us/step - loss: 0.0225 - acc: 0.8657 - weighted_accuracy: 0.8599 - val_loss: 0.3283 - val_acc: 0.8458 - val_weighted_accuracy: 0.8422
Epoch 5/500
365054/365054 [==============================] - 85s 233us/step - loss: 0.0221 - acc: 0.8694 - weighted_accuracy: 0.8640 - val_lo